In [13]:
import boto3
from typing import List, Optional, Dict, Any, Tuple
import brokit as bk
class NovaBedrock(bk.LM):
    def __init__(
            self, 
            model_name:str, 
            temperature:float=0.0, 
            top_p:float=1.0, 
            seed:int=555,
            region_name:str='us-east-1', 
            access_key_id: Optional[str] = None,
            secret_access_key: Optional[str] = None,
            session_token: Optional[str] = None
        ):
        super().__init__(model_name=model_name, model_type=bk.ModelType.CHAT)
        self.temperature = temperature
        self.top_p = top_p
        self.seed = seed

        self.client = boto3.client(
            'bedrock-runtime',
            region_name=region_name,
            aws_access_key_id=access_key_id,
            aws_secret_access_key=secret_access_key,
            aws_session_token=session_token
        )

    def pack_message(self, messages:List[bk.Message])->Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
        system_prompt = []
        _messages = []
        for message in messages:
            if message.role=="system":
                system_prompt.append({"text": message.content})
            else:
                _messages.append({
                    "role": message.role,
                    "content": [
                        {"text": message.content}
                    ]
                })
        return system_prompt, _messages

    def request(self, prompt:Optional[str]=None, messages:Optional[List[bk.Message]]=None, **kwargs) -> dict:
        if messages:
            system_prompt, _messages = self.pack_message(messages)
        else:
            _messages = [{"role": "user", "content": [
                {"text": prompt}
            ]}]
            system_prompt = None
        request_params = {
            "modelId": self.model_name,
            "messages": _messages,
            "inferenceConfig": {
                "temperature": self.temperature,
                "topP": self.top_p,
                # "maxTokens": kwargs.get("max_tokens", 2048),
            }
        }
        if system_prompt:
            request_params["system"] = system_prompt
        response = self.client.converse(**request_params)
        return response
    
    def parse_response(self, original_response: dict) -> bk.ModelResponse:
        message = original_response["output"]["message"]
        input_tokens = original_response["usage"]["inputTokens"]
        output_tokens = original_response["usage"]["outputTokens"]
        return bk.ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"][0]["text"],
            usage=bk.Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

In [14]:
nova = NovaBedrock(model_name="us.amazon.nova-lite-v1:0")

In [15]:
nova.request(prompt="Hello how are you?")

{'ResponseMetadata': {'RequestId': '852764ee-ea98-4cad-89b7-5af58b0733ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Feb 2026 11:55:27 GMT',
   'content-type': 'application/json',
   'content-length': '354',
   'connection': 'keep-alive',
   'x-amzn-requestid': '852764ee-ea98-4cad-89b7-5af58b0733ad'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': "Hello! I'm doing well, thank you for asking. How can I assist you today? If you have any questions or need help with something, feel free to let me know."}]}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 5, 'outputTokens': 39, 'totalTokens': 44},
 'metrics': {'latencyMs': 469}}

In [ ]:
nova(prompt="Hello how are you?")

ModelResponse(model_name='us.amazon.nova-lite-v1:0', model_type=<ModelType.CHAT: 'chat'>, response="Hello! I'm doing well, thank you for asking. How can I assist you today? If you have any questions or need help with something, feel free to let me know.", usage=Usage(input_tokens=5, output_tokens=39), response_ms=639.3004999845289, cached=False, metadata=None, request=None, parsed_response=None)